In [1]:
import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
np = numpy
plt = pyplot
from matplotlib.pyplot import *
import os
from scipy.optimize import curve_fit
from IPython.display import display, clear_output
from IPython.core.pylabtools import figsize, getfigs
from pylab import *
from numpy import *
import pandas as pd
import ipywidgets as widgets
import pybroom as br
import lmfit
import numpy as np
import numpy as np
from IPython.display import Math
from ipywidgets import interact,Dropdown
import ipywidgets as widgets # Widget definitions
from IPython.display import display # Used to display widgets in the notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import csv
from numpy import genfromtxt
import glob
import random
import sys
np.set_printoptions(threshold=sys.maxsize)

In [2]:
def func(x, ystress, eta_bg, power):
    return ystress + eta_bg * (x ** power)

In [3]:
N = '02'
Batch = int(N)

In [4]:
Train_data = np.transpose((0,0,0,0,0,0,0,0,0,0))

A = genfromtxt('../../HFData/ExpT25C/Batch'+N+'.txt')
j=0
while j<len(A):
    if A[j][4] == 1 and A[j][5] == 25 and A[j][6] == 0.1 and abs(A[j][7] - 25) < 1:
        Train_data = np.vstack((Train_data,A[j]))
    j=j+1
    
Train_data = np.delete(Train_data, (0), axis=0)

In [5]:
xdata=Train_data.reshape((-1,1))[8::10]
ydata=Train_data.reshape((-1,1))[9::10]

ydata = (ydata[10:]*xdata[10:]).reshape((32,))
xdata = xdata[10:].reshape((32,))

popt, pcov = curve_fit(func, xdata, ydata, maxfev=10000, bounds=(0.01, inf))
Sig_Yield = popt[0]
Eta_BG = popt[1]
Power = popt[2]

In [6]:
Test_data = np.transpose((0,0,0,0,0,0,0,0,0,0))

A = genfromtxt('../../HFData/ExpRm2/Batch'+N+'.txt')
j=0
while j<len(A):
    if A[j][4] == 1 and A[j][5] == 25 and A[j][6] == 0.1:
        Test_data = np.vstack((Test_data,A[j]))
    j=j+1

Test_data = np.delete(Test_data, (0), axis=0)

In [7]:
ratio = (Test_data[0][9])/(Test_data[89][9])
Sig_Yield_S1 = Sig_Yield# * np.log10(ratio)
Eta_BG_S1 = Eta_BG / ratio
Power_S1 = Power# * np.log10(ratio)

In [8]:
Train_data = np.transpose((0,0,0,0,0,0,0,0,0,0))

A = genfromtxt('../../HFData/ExpT25C/Batch'+N+'.txt')
j=0
while j<len(A):
    if A[j][4] == 5 and A[j][5] == 25 and A[j][6] == 0.1 and abs(A[j][7] - 25) < 1:
        Train_data = np.vstack((Train_data,A[j]))
    j=j+1
    
Train_data = np.delete(Train_data, (0), axis=0)

In [9]:
xdata=Train_data.reshape((-1,1))[8::10]
ydata=Train_data.reshape((-1,1))[9::10]

ydata = (ydata[10:]*xdata[10:]).reshape((32,))
xdata = xdata[10:].reshape((32,))

popt, pcov = curve_fit(func, xdata, ydata, maxfev=10000, bounds=(0.1, inf))
Sig_Yield = popt[0]
Eta_BG = popt[1]
Power = popt[2]

In [10]:
Sig_Yield_S2 = Sig_Yield# * np.log10(ratio)
Eta_BG_S2 = Eta_BG / ratio
Power_S2 = Power# * np.log10(ratio)

In [11]:
Sig_Yield_S3 = Sig_Yield_S1 + 2.25*(Sig_Yield_S2-Sig_Yield_S1)
Eta_BG_S3 = Eta_BG_S1 + 2.25*(Eta_BG_S2-Eta_BG_S1)
Power_S3 = abs(Power_S1 + 2.25*(Power_S2-Power_S1))

In [12]:
Gama = np.power(10,numpy.linspace(-2,2,420))
ST = Sig_Yield_S3 + Eta_BG_S3 * (Gama ** Power_S3)
Vis = (ST / Gama)

In [13]:
Train_data = np.vstack((Train_data,Train_data,Train_data,Train_data,Train_data,Train_data,Train_data,Train_data,Train_data,Train_data))

j=0
while j<len(Train_data):
    Train_data[j][9] = Vis[j]
    Train_data[j][8] = Gama[j]    
    Train_data[j][7] = 40
    Train_data[j][4] = 10
    j=j+1

In [14]:
#plt.plot(Gama,Vis)
#plt.xscale('log')
#plt.yscale('log')
#plt.show()

In [15]:
np.savetxt('Batch' + N +'.txt', Train_data)